In [1]:
import pyTigerGraph as tg
import dgl
import networkx as nx
import torch
from py_scripts.gat import GAT
import torch.nn as nn
import torch.nn.functional as F
import py_scripts.cfg as cfg
import gensim
import time
import numpy as np

In [2]:
numEpochs = 1000
learningRate = 0.01
wantedTopic = "Saxophone"
unwantedTopic = "Falkland_Islands" #In future, determine automatically through the inverse of PageRank/other centrality algo

In [3]:
conn = tg.TigerGraphConnection(ipAddress="https://wikipediagraph.i.tgcloud.us", apiToken=cfg.token, graphname="WikipediaGraph") # connection to TigerGraph database

articleToNum = {} # translation dictionary for article name to number (for dgl)
numToArticle = {} # translation dictionary for number to article name
i = 0
def createEdgeList(result): # returns tuple of number version of edge
    global i
    if result["article1"] in articleToNum:
        fromKey = articleToNum[result["article1"]]
    else:
        articleToNum[result["article1"]] = i
        numToArticle[i] = result["article1"]
        fromKey = i
        i+=1
    if result["article2"] in articleToNum:
        toKey = articleToNum[result["article2"]]
    else:
        articleToNum[result["article2"]] = i
        numToArticle[i] = result["article2"]
        toKey = i
        i+=1
    return (fromKey, toKey)
    
edges = [createEdgeList(thing) for thing in conn.runInstalledQuery("tupleArticle", {})["results"][0]["list_of_article_tuples"]] # creates list of edges


In [4]:
wantedNum = articleToNum[wantedTopic]
unwantedNum = articleToNum[unwantedTopic]

g = nx.DiGraph()

g.add_edges_from(edges)

G = dgl.DGLGraph(g)

print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())

We have 4588 nodes.
We have 119881 edges.


In [5]:
d2v_model = gensim.models.doc2vec.Doc2Vec.load("doc2vec.model")
vectors = []
for article in articleToNum:
    try: 
        words = [] 
        with open(("plaintext_articles/"+article+".txt"), "r") as f: 
            words = f.read().lower().split() 
        vectors.append(list(d2v_model.infer_vector(words)))
    except:
        vectors.append([0 for i in range(300)])

G.ndata["feat"] = torch.tensor(vectors)

print(G.nodes[2].data['feat'])

tensor([[-13.4399,  -8.4157,  -5.7430,   5.0115,  -0.2688,  -1.9023,   0.6301,
          12.0454,  17.6172,   2.6363,   2.7329,  -2.9679, -32.0563, -14.5386,
           2.6261, -17.6840, -21.5968, -11.3046, -20.4120,   0.8874, -26.9279,
          -4.9758, -12.1565,  26.8152,  17.8495,  -2.4941,   7.8032,  17.2047,
          28.4754,  -8.9398, -23.7910,   3.0749,  -0.7522,  22.6601, -16.5475,
          -3.4500,  12.8002, -11.0542,  17.8362,  32.9645,  -1.9119,  -6.0420,
          10.4199,   6.0184, -10.5404, -16.5472, -12.3625,  -6.7955,   5.1744,
          35.4032,   1.7788,  19.9572,  -3.4260,  12.6115,   1.5618,  16.5668,
          -3.7526,   3.9297,  -9.9369,  -4.5472,  28.8338,  34.5560,  15.3711,
          -2.1632,   3.5442,  10.2373,   6.8024, -13.5585, -35.7496, -14.3184,
         -11.7207,  -1.1009,  -1.7906,  -0.1248, -11.6610,  10.2103,  -1.4551,
         -13.9517, -19.5545, -13.3866, -22.8139, -20.6193,   4.9487,  30.1665,
          -1.0930,   9.3571,  13.6123, -12.6326,  -7

In [8]:
dur = []
for epoch in range(30):
    if epoch >= 3:
        t0 = time.time()

    logits = net(G.ndata["feat"])
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[mask], labels[mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
        epoch, loss.item(), np.mean(dur)))

NameError: name 'mask' is not defined

In [6]:
net = GAT(G, in_dim=300, hidden_dim=8, out_dim=2, num_heads=2)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)